In [ ]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T
from torchvision import io
import torchutils as tu
import json
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
def plot_history(history, grid=True):
    fig, ax = plt.subplots(1,2, figsize=(14,5))
    
    ax[0].plot(history['train_losses'], label='train loss')
    ax[0].plot(history['valid_losses'], label='valid loss')
    ax[0].set_title(f'Loss on epoch {len(history["train_losses"])}')
    ax[0].grid(grid)
    ax[0].set_ylim((0, max(history['train_losses'] + history['valid_losses']) + .1))
    ax[0].legend()
    
    ax[1].plot(history['train_accs'], label='train acc')
    ax[1].plot(history['valid_accs'], label='valid acc')
    ax[1].set_title(f'Accuracy on epoch {len(history["train_losses"])}')
    ax[1].grid(grid)
    ax[1].set_ylim((0, 1))
    ax[1].legend()
    
    plt.show()

In [ ]:
def fit_model(
        model: nn.Module, 
        epochs: int, 
        optimizer: torch.optim.Optimizer, 
        history=None
    ):
    history = history or {
        'train_accs': [],
        'train_losses': [],
        'valid_accs': [],
        'valid_losses': [],
    }

    for epoch in range(1, epochs + 1):
        model.train()
        batch_losses = []
        batch_accs = []
        
        for samples, labels in train_loader:
            samples = samples.to(device)
            labels = labels.to(device)
            
            y_pred = model(samples)
            loss = criterion(y_pred, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            batch_losses.append(loss.item())
            batch_accs.append((y_pred.argmax(dim=1) == labels).float().mean().item())

        train_loss = np.mean(batch_losses)
        train_acc  = np.mean(batch_accs)
        history['train_losses'].append(train_loss)
        history['train_accs'].append(train_acc)

        model.eval()
        batch_losses = []
        batch_accs = []
        with torch.no_grad():
            for samples, labels in valid_loader:
                samples = samples.to(device)
                labels = labels.to(device)
                y_pred = model(samples)
                loss = criterion(y_pred, labels)
                batch_losses.append(loss.item())
                batch_accs.append((y_pred.argmax(dim=1) == labels).float().mean().item())

        valid_loss = np.mean(batch_losses)
        valid_acc  = np.mean(batch_accs)
        history['valid_losses'].append(valid_loss)
        history['valid_accs'].append(valid_acc)

        # # [MLflow] Логируем метрики
        # if use_mlflow:
        #     # epoch – номер шага (можно указывать step=epoch)
        #     mlflow.log_metric("train_loss", train_loss, step=epoch)
        #     mlflow.log_metric("train_acc",  train_acc,  step=epoch)
        #     mlflow.log_metric("valid_loss", valid_loss, step=epoch)
        #     mlflow.log_metric("valid_acc",  valid_acc,  step=epoch)

        print(
            f"Epoch {epoch}/{epochs} | "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
            f"Val Loss: {valid_loss:.4f}, Val Acc: {valid_acc:.4f}"
        )

    plot_history(history)
    return history

In [ ]:
from torchvision.models import MobileNet_V3_Small_Weights

In [ ]:
model = torchvision.models.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)

In [ ]:

model.classifier[3] = nn.Linear(1024,1)
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True
for param in model.features[12].parameters():
    param.requires_grad = True
for param in model.features[11].parameters():
    param.requires_grad = True
for param in model.features[10].parameters():
    param.requires_grad = True
for param in model.features[9].parameters():
    param.requires_grad = True
for param in model.features[8].parameters():
    param.requires_grad = True
for param in model.features[7].parameters():
    param.requires_grad = True
for param in model.features[6].parameters():
    param.requires_grad = True   
for param in model.features[5].parameters():
    param.requires_grad = True  
for param in model.features[4].parameters():
    param.requires_grad = True  
for param in model.features[3].parameters():
    param.requires_grad = True  
for param in model.features[2].parameters():
    param.requires_grad = True  
for param in model.features[1].parameters():
    param.requires_grad = True  

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
history = fit_model(model, 10, optimizer=optimizer) 